#### Portfolio optimization of crypto currencies:
a popular technique that financial and investment advisors use to help ensure that their clients are taking appropriate risk to meet their financial objectives.

Section 1: Focusing on collecting the data necessary for analysis. Closing price for top10 coins according to marketcap. Use API CoinGecko to grab daily closing price in USD for 365 days during bear market (from 01.01.2019 - 31.12.2019).
BTC ETH XRP ADA LTC BNB BCH XLM BSV EOS

Section 2: Preparing the data to calculate a few different metrics: expected returns, expected volatility, and the sharpe ratio.

Section 3: Building a simulation that will randomly generate results that we can use to help to determine the optimal results for a wide range of portfolios.

Section 4: Taking this simulation, plot them using matplotlib, using scipy library to run an optimization algorthim that will return the best weights for our simulation


In [1]:
import pandas as pd
import requests
import json
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.optimize as sci_plt

from pprint import pprint
from sklearn.preprocessing import StandardScaler

# Set some display option for Pandas

pd.set_option("display.max_colwidth", None)
pd.set_option("expand_frame_repr", False)

### Section 1 
Grabbing the data from CoinGecko

Firstly to get the correct IDs

In [2]:
base = "https://api.coingecko.com/api/v3/"
url = base + "/coins/list"
response = requests.get(url)
results = response.json()
df_list = pd.DataFrame(results)
# df_list[df_list['symbol'].str.match("bnb")] : binancecoin
# df_list[df_list['symbol'].str.match("bch")]# : bitcoin-cash
# df_list[df_list['symbol'].str.match("bsv")] : bitcoin-cash-sv

Secondly to generate a dataframe of all the prices from 01.Jan.19 to 31.12.19

In [10]:
'''all_coins = []
base = "https://api.coingecko.com/api/v3/"
ids = ["bitcoin", "ethereum", "ripple", "cardano", "litecoin", "binancecoin", "bitcoin-cash", "stellar", "bitcoin-cash-sv", "eos"]
for coin in ids:
    url = base + "/coins/"+ coin + "/market_chart/range"
    param_dict = {"id":coin, "vs_currency":"usd", "from":"1546297200", "to":"1577833200"}
    response = requests.get(url, params=param_dict)
    results = response.json()
    df = pd.DataFrame(results["prices"], columns=["time", coin])
    df["time"] = pd.to_datetime(df["time"], unit="ms")
    # df_all = pd.merge(df_all, df, how="left", on="time")
    all_coins.append(df)

all_coins_merged = reduce(lambda df1, df2: pd.merge(df1,df2, how="left", on="time"), all_coins)
all_coins_merged.set_index("time", inplace=True)

all_coins_merged.to_csv("all_coins_2019.csv")
# round({"bitcoin":0, "ethereum":0, "ripple":4, "cardano":4, "litecoin":1, "binancecoin":1, "bitcoin-cash":1, "stellar":4, "bitcoin-cash-sv":1, "eos":2})
'''

### Section 2
When we compare the performance of an investment, we should take both return and risk into consideration. Most investors aren't okay with taking on high-levels of risk, so the goal is to find the best ratio of risk vs returns. Ideally, we are taking the most risk we are comfortable with and attempting to maximize those returns. That naturally leads to an important question, "How do we measure risk?". There are different ways to measure risk, in this workbook, we will use a very popular metric, the Sharpe Ratio. The Sharpe Ratio, is used as a measure for calculating risk-adjusted return and has been the industry standard for such calculations. The Sharpe Ratio allows us to quantify the relationship the average return earned in excess of the risk-free rate per unit of volatility or total risk.

Mathematically, we define the Sharpe Ratio as the following:

$$\begin{equation*}
\text{Sharpe Ratio} = \frac{(R_p - R_f)}{\sigma_p}
\end{equation*}$$
Where:

$
\begin{align}
\ R_p = \text{Return of Portfolio} \\
\ R_f = \text{Risk-Free Rate} \\
\ \sigma_p = \text{Standard Deviation of Portfolio's Excess Return} \\
\end{align}
$

To calculate the expected returns, we use the following formula:

$$\begin{equation*}
\ R_p = (w_{1}r_{1}) + (w_{2}r_{2}) + \cdots + (w_{n}r_{n})
\end{equation*}$$
Where:

$
\begin{align}
\ r_{i} = \text{Return of Security i} \\
\ w_{i} = \text{Weight of Security i} \\
\end{align}
$

To calculate the standard deviation of the protfolio, we use the following formula:

$$\begin{equation*}
\ \sigma_p = \sqrt{(w_{i}^2 \sigma_i^2) + (w_{j}^2 \sigma_j^2) + (2w_{j}w_{i} p_{i,j} \sigma_i \sigma_j)}
\end{equation*}$$
Where:

$
\begin{align}
\ \sigma_{i}  = \text{Standard Deviation of Returns for Security i} \\
\ w_{i}  = \text{Weight of Security i} \\
\ p_{i,j}  = \text{Correlation Coefficent between the returns of asset i and asset j} \\
\end{align}
$

In [14]:
# read csv file
all_coins_merged = pd.read_csv("all_coins_2019.csv")
all_coins_merged.set_index("time", inplace=True)
all_coins_merged.head()

,bitcoin,ethereum,ripple,cardano,litecoin,binancecoin,bitcoin-cash,stellar,bitcoin-cash-sv,eos
time,,,,,,,,,,
2019-01-01,3692.531566,131.458725,0.349173,0.040537,29.860457,6.082209,148.840366,0.111091,83.887480,2.523546
2019-01-02,3794.264254,138.144802,0.360089,0.041910,31.424784,5.987467,162.447095,0.114158,90.646882,2.616709
2019-01-03,3872.172184,152.860453,0.370770,0.044715,32.990775,6.057147,168.532933,0.117350,93.163366,2.833973
2019-01-04,3776.525808,146.730599,0.353943,0.042007,31.442759,5.800076,159.162407,0.112371,87.165476,2.630978
2019-01-05,3815.268080,153.056567,0.355352,0.043254,31.950440,5.967081,159.770460,0.113637,86.628600,2.686171


In [26]:
# log return over time
# calculate the log returns
log_return = np.log(1 + all_coins_merged.pct_change())

# generate random weights
random_weights = np.array(np.random.random(len(ids)))

# generate rebalanced random weights, sum up to 1
rebalanced_weights = random_weights / np.sum(random_weights)
rebalanced_weights

# calculate the expected returns and annualised it 365
exp_ret = np.sum((log_return.mean() * rebalanced_weights)*365)

# calculate the expected volatility and annualised it
exp_vol = np.sqrt(
    np.dot(
        rebalanced_weights.T,
        np.dot(
            log_return.cov() * 365,
            rebalanced_weights
        )  
    )
)

# calculate the sharpe ratio. Assumed that the risk free rate is 1%

sharpe_ratio = (exp_ret - 0.01) / exp_vol

# put the weights into a data frame for a better overview
weights_df = pd.DataFrame(data={
            "random_weights" : random_weights,
            "rebalanced_weights" : rebalanced_weights
})

print("")
print("*"*100)
print("Portfolio Weights:")
print("*"*100)
print(weights_df)
print("-"*100)

# put the other metrics into a data frame for a better overview
metrics_df = pd.DataFrame(data={
            "Expected Portfolio Returns" : exp_ret,
            "Expected Portfolio Volatility" : exp_vol,
            "Portfolio Sharpe Ratio" : sharpe_ratio
}, index=[0])
print("")
print("*"*100)
print("Portfolio Metrics:")
print("*"*100)
print(metrics_df)
print("-"*100)


****************************************************************************************************
Portfolio Weights:
****************************************************************************************************
   random_weights  rebalanced_weights
0        0.188962            0.032044
1        0.056329            0.009552
2        0.731353            0.124020
3        0.993123            0.168410
4        0.935611            0.158658
5        0.888457            0.150661
6        0.415244            0.070416
7        0.293259            0.049730
8        0.696064            0.118036
9        0.698641            0.118473
----------------------------------------------------------------------------------------------------

****************************************************************************************************
Portfolio Metrics:
****************************************************************************************************
   Expected Portfolio Returns  Expect

### Section 3
Portfolio Optimization: Monte Carlo

By calculating the expected returns, the expected volatility and using the Sharpe Ratio to quantify how well the portfolio is allocated based on a risk perspective. The question is, "if we know that we want to get a higher sharpe ratio, what is the portfolio allocation we need to achieve this?"

There are two ways to approach this question. 
1) We could test a bunch of different random allocations and see which ones produces the highest Sharpe Ratio.
2) We could use mathematical optimization defined by some constrains to arrive at the optimial allocation -> Monte Carlo simulations

Monte Carlo simulations are used to model the probability of different outcomes in a process that cannot easily be predicted due to the intervention of random variables. It is a technique used to understand the impact of risk and uncertainty in prediction and forcasting models.

To do this, we can run the process we performed a couple thousands of time and store the results of each run in a data frame. once we've completed each run, we will find the allocations that produce the highest Sharpe Ratio and lowest Volatility.

In [ ]:
# define the number of times that we want to run the simulation

num_of_simulations = 10000

# set the weight array by using numpy zeros

all_weights_array = np.zeros(num_of_simulations, len(ids))

# set the returns array by using numpy zeros

ret_array = np.zeros(num_of_simulations)

# set the volatility array by using numpy zeros
v_array = np.zeros(num_of_simulations)

# set the sharpe ratio array by using numpy zeros

sharpe_array = np.zeros(num_of_simulations)

# start the simulation
for trial in range(num_of_simulations):
    
    # firt, calculate the weights
    weights = np.array(np.random.random(len(ids)))
    weights = weights / np.sum(weights)
    
    # add the weights to the "weights_array"
    all_weights_array[trial, :] = weights
    
    # calculate the expected log returns
    ret_array[trial] = np.sum((log_return.mean() * weights)*365)
    
    # calculate the volatility
    vol_array[trial] = np.sqrt(
      np.dot(weights.T, np.dot(log_return.cov() * 365, weights))
    )
    
    # calculate the sharpe ratio
    sharpe_array[trial] = ret_array[trial] / vol_array[trial]